# Agent-based RAG system for Nerve API documentation

## Imports

In [5]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from typing import Callable, List, Tuple, Union
import sys

# Add path
sys.path.append('../..')

from inswitch.llm.model import get_openai_model_config
from inswitch.agent.basic import get_llm_agent, get_fixed_reply_agent
from inswitch.agent.ragagent import RagAgent

## Load all relevant Nerve API documents to be added to RAG

In [6]:
doc_1 = "https://docs.nerve.cloud/developer_guide/dna/"
doc_2 = "https://docs.nerve.cloud/developer_guide/networking/"
doc_3 = "https://docs.nerve.cloud/developer_guide/codesys/"
doc_4 = "https://docs.nerve.cloud/developer_guide/ms-api/api-pointers/"
doc_5 = "https://docs.nerve.cloud/developer_guide/ms-api/api-examples/"

## RAG-enhanced agent 

In [7]:
nerve_rag_agent = RagAgent(
    name = "rag_agent", 
    docs_path = [doc_1, doc_2, doc_3, doc_4, doc_5],
    max_internal_turns=5
)


In [8]:
moderator = get_fixed_reply_agent(
    name="moderator",
    reply = ""
)

deploy_plan = """Deploy the following workloads for Machine: M00001 (Type: MTC)

      - Workload: ngix, Version: ngix_27 (1.27.2)
      - Workload: nodejs, Version: nodejs_23 (23.2.0)
      - Workload: alarm_record
      - Workload: mqtt_sender
      
"""

deployment_plan_announcer = get_fixed_reply_agent(
    'deployment_plan_announcer',
    reply=deploy_plan
)

chat_result = moderator.initiate_chats(
    [
        {
            "recipient": deployment_plan_announcer,
            "message": "What should be deployed? on which machine?",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": nerve_rag_agent,
            "message": "Provide context",
            "max_turns": 1,
            "summary_method": "last_msg",
        }
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to deployment_plan_announcer):

What should be deployed? on which machine?

--------------------------------------------------------------------------------
deployment_plan_announcer (to moderator):

Deploy the following workloads for Machine: M00001 (Type: MTC)

      - Workload: ngix, Version: ngix_27 (1.27.2)
      - Workload: nodejs, Version: nodejs_23 (23.2.0)
      - Workload: alarm_record
      - Workload: mqtt_sender
      


--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to rag_agent):

Provide context
Context: 
Deploy the following workloads for Machin